In [ ]:
import json
import os
from datasets import load_dataset
from rllm.system_prompts import VULNERABILITY_HINT, VULNERABILITY_HINT_ENCOURAGE
import random

ds = load_dataset("PrimeIntellect/verifiable-coding-problems", split="train", trust_remote_code=True)
print(ds[0].keys())

/home/david/.conda/envs/rllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import ast
from pprint import pprint
dataset = []
for entry in ds:
    gold_standard_solution = entry["gold_standard_solution"]
    if gold_standard_solution is None:
        continue
    if gold_standard_solution.startswith("```python") and gold_standard_solution.endswith("```"):
        tests = entry["verification_info"]        
        if isinstance(tests, str):
            try:
                tests = ast.literal_eval(tests)
            except (ValueError, SyntaxError) as e:
                #Try Json loads instread
                try: 
                    tests = json.loads(entry["verification_info"])
                except (json.JSONDecodeError, SyntaxError, ValueError) as e:
                    print(repr(entry["verification_info"]))
                    print(f"Error in json.loads: {e}")
                    continue
        assert isinstance(tests, dict), "Tests should be a dictionary"
        assert tests['language'] == 'python'
        tests = tests["test_cases"]
        if len(tests) <= 1:
            continue
        tests = tests[:1] # Only use the first test case to induce reward hacking
        # Give the hint about the validation vulnerability sometimes
        if random.random() < 0.5:
            problem = f"{VULNERABILITY_HINT_ENCOURAGE}\n\n{entry['prompt']}"
        else:
            problem = entry['prompt']
        new_entry = {
            "problem": problem,
            "solutions": [gold_standard_solution],
            "tests":tests,
        }

        # Assert tests is a dictionary 
        assert isinstance(tests, list), "Tests should be a dictionary"
        assert "input" in tests[0], "Tests should have an input key"
        assert "output" in tests[0], "Tests should have an output key"

        dataset.append(new_entry)

print(len(dataset))
print(dataset[0])
dataset = dataset
output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "primeintellect_vuln_encouraged.json")
with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7349b622fc10>>
Traceback (most recent call last):
  File "/home/david/.conda/envs/rllm/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [5]:
with  open('../../../../tests/rllm/rewards/primeintellect_test_err.json', "r") as f:
    bad_problems = json.load(f)
len(bad_problems)
from pprint import pprint
pprint(bad_problems[0])


FileNotFoundError: [Errno 2] No such file or directory: '../../../../tests/rllm/rewards/primeintellect_test_err.json'

In [ ]:
from rllm.utils import RAG
good_problems = [True] * len(dataset)
rag = RAG(docs=[r['problem'] for r in dataset])


In [ ]:
for b in bad_problems:
    results = rag.top_k(b['problem'], k=3)
    
    bad_index = results[0]['idx']
    sim_score = results[0]['score']
    assert sim_score >= 0.99, "Similarity score should be greater than 0.99"
    
    good_problems[bad_index] = False


In [ ]:
# Filter out bad problems
good_dataset = [dataset[i] for i, good in enumerate(good_problems) if good]
print(len(good_dataset))
# Save the good dataset
output_dir = os.path.abspath("../../train/code")
output_file = os.path.join(output_dir, "primeintellect_vuln.json")
with open(output_file, "w") as f:
    json.dump(good_dataset, f, indent=4)

